<a href="https://colab.research.google.com/github/faisal12212092/DDoS-Attack-Detection-Using-Deep-Learning-Research-report-/blob/main/simple_RNN_Xvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [ ]:
path = "/content/drive/MyDrive/DDOS DATASETS/smtnormalizeddata.csv"
df = pd.read_csv(path)

In [ ]:
df.dropna(inplace=True)
features = df.drop('Label', axis=1)
target = df['Label']

In [ ]:
def create_sequences(data, labels, sequence_length):
    sequences = []
    sequence_labels = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
        sequence_labels.append(labels[i + sequence_length])
    return np.array(sequences), np.array(sequence_labels)

In [ ]:
sequence_length = 10

X, y = create_sequences(features.values, target.values, sequence_length)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

cv_accuracy = []
cv_precision = []
cv_recall = []
cv_f1 = []
cv_cm = []

#K-fold cross-validation
for train_index, val_index in kfold.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    model = Sequential()
    model.add(SimpleRNN(50, return_sequences=True, input_shape=(sequence_length, X_train.shape[2])))
    model.add(SimpleRNN(50))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(X_train, y_train, epochs=5, batch_size=150, validation_data=(X_val, y_val))

Epoch 1/5
1366/1366 [==============================] - 35s 23ms/step - loss: 0.0109 - accuracy: 0.9977 - val_loss: 0.0055 - val_accuracy: 0.9988
Epoch 2/5
1366/1366 [==============================] - 30s 22ms/step - loss: 0.0038 - accuracy: 0.9993 - val_loss: 0.0035 - val_accuracy: 0.9994
Epoch 3/5
1366/1366 [==============================] - 33s 24ms/step - loss: 0.0031 - accuracy: 0.9994 - val_loss: 0.0032 - val_accuracy: 0.9993
Epoch 4/5
1366/1366 [==============================] - 30s 22ms/step - loss: 0.0028 - accuracy: 0.9995 - val_loss: 0.0051 - val_accuracy: 0.9991
Epoch 5/5
1366/1366 [==============================] - 31s 22ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 0.0022 - val_accuracy: 0.9995
Epoch 1/5
1366/1366 [==============================] - 34s 23ms/step - loss: 0.0145 - accuracy: 0.9957 - val_loss: 0.0039 - val_accuracy: 0.9993
Epoch 2/5
1366/1366 [==============================] - 32s 24ms/step - loss: 0.0042 - accuracy: 0.9992 - val_loss: 0.0035 - val_ac

In [ ]:
    y_pred = model.predict(X_val)
    y_pred = (y_pred > 0.5).astype(int)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    cm = confusion_matrix(y_val, y_pred)

    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    cv_f1.append(f1)
    cv_cm.append(cm)

1601/1601 [==============================] - 10s 6ms/step


In [ ]:
avg_accuracy = np.mean(cv_accuracy)
avg_precision = np.mean(cv_precision)
avg_recall = np.mean(cv_recall)
avg_f1 = np.mean(cv_f1)
avg_cm = np.mean(cv_cm, axis=0)

print(f'Average Accuracy: {avg_accuracy}')
print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')
print(f'Average F1 Score: {avg_f1}')
print('Average Confusion Matrix:')
print(avg_cm.astype(int))

Average Accuracy: 0.9974027495703797
Average Precision: 0.9951450654445179
Average Recall: 0.9996878657822864
Average F1 Score: 0.9974112929910272
Average Confusion Matrix:
[[25453   125]
 [    8 25622]]
